<a href="https://colab.research.google.com/github/AlexeyProvorov/Generative/blob/master/Attention_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Импортируем необходимые библиотеки
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Устанавливаем устройство для вычислений (CPU или GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# === 1. Подготовка данных ===

# Наборы предложений на английском и французском языках
english_sentences = [
    "hello how are you",
    "i am fine thank you",
    "what is your name",
    "my name is john",
    "nice to meet you"
]

french_sentences = [
    "bonjour comment ça va",
    "je vais bien merci",
    "quel est ton nom",
    "mon nom est john",
    "ravi de vous rencontrer"
]

# Функция для создания словаря из списка предложений
def build_vocab(sentences):
    # Начинаем с специальных токенов
    vocab = {"<PAD>": 0, "<SOS>":1, "<EOS>":2, "<UNK>":3}
    index = 4  # Начальный индекс для новых слов
    for sentence in sentences:
        for word in sentence.split():
            if word not in vocab:
                vocab[word] = index
                index += 1
    return vocab

# Создаём словари для английского и французского языков
english_vocab = build_vocab(english_sentences)
french_vocab = build_vocab(french_sentences)

# Функция для преобразования предложения в последовательность индексов
def sentence_to_indices(sentence, vocab):
    # Разбиваем предложение на слова и заменяем их на индексы
    indices = [vocab.get(word, vocab["<UNK>"]) for word in sentence.split()]
    # Добавляем токен конца предложения
    indices.append(vocab["<EOS>"])
    return indices

# Преобразуем все предложения в последовательности индексов
english_sequences = [sentence_to_indices(sentence, english_vocab) for sentence in english_sentences]
french_sequences = [sentence_to_indices(sentence, french_vocab) for sentence in french_sentences]

# === 2. Создание датасета и загрузчика данных ===

# Класс для датасета перевода
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, input_sequences, target_sequences):
        self.input_sequences = input_sequences  # Список входных последовательностей
        self.target_sequences = target_sequences  # Список целевых последовательностей

    def __len__(self):
        return len(self.input_sequences)  # Возвращаем количество образцов

    def __getitem__(self, idx):
        return self.input_sequences[idx], self.target_sequences[idx]  # Возвращаем пару (вход, цель)

# Функция для объединения батчей и выравнивания последовательностей
def collate_fn(batch):
    input_seqs, target_seqs = zip(*batch)

    # Находим максимальную длину в батче для входных и целевых последовательностей
    max_input_len = max(len(seq) for seq in input_seqs)
    max_target_len = max(len(seq) for seq in target_seqs)

    # Заполняем последовательности токеном <PAD> до максимальной длины
    padded_inputs = [seq + [english_vocab["<PAD>"]] * (max_input_len - len(seq)) for seq in input_seqs]
    padded_targets = [seq + [french_vocab["<PAD>"]] * (max_target_len - len(seq)) for seq in target_seqs]

    # Преобразуем списки в тензоры
    return torch.tensor(padded_inputs, dtype=torch.long), torch.tensor(padded_targets, dtype=torch.long)

# Создаём экземпляр датасета
dataset = TranslationDataset(english_sequences, french_sequences)

# Создаём загрузчик данных
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# === 3. Определение модели ===

# Параметры модели
INPUT_DIM = len(english_vocab)  # Размер словаря входного языка
OUTPUT_DIM = len(french_vocab)  # Размер словаря целевого языка
ENC_EMB_DIM = 256  # Размерность эмбеддингов кодера
DEC_EMB_DIM = 256  # Размерность эмбеддингов декодера
HID_DIM = 512      # Размерность скрытого состояния RNN

# Кодер
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)  # Слой эмбеддингов
        self.rnn = nn.GRU(emb_dim, hid_dim, batch_first=True)  # RNN (GRU)

    def forward(self, src):
        # src: [batch_size, src_len]
        embedded = self.embedding(src)  # [batch_size, src_len, emb_dim]
        outputs, hidden = self.rnn(embedded)  # outputs: [batch_size, src_len, hid_dim], hidden: [1, batch_size, hid_dim]
        return outputs, hidden  # Возвращаем все выходы и последнее скрытое состояние

# Механизм внимания
class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        # Линейный слой для вычисления энергии внимания
        self.attn = nn.Linear(hid_dim * 2, hid_dim)
        # Вектор контекстного веса
        self.v = nn.Parameter(torch.rand(hid_dim))

    def forward(self, hidden, encoder_outputs):
        # hidden: [batch_size, hid_dim]
        # encoder_outputs: [batch_size, src_len, hid_dim]
        src_len = encoder_outputs.shape[1]

        # Повторяем скрытое состояние для каждого шага времени
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)  # [batch_size, src_len, hid_dim]

        # Вычисляем энергию внимания
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # [batch_size, src_len, hid_dim]

        # Переставляем измерения для умножения
        energy = energy.permute(0, 2, 1)  # [batch_size, hid_dim, src_len]

        # Повторяем контекстный вектор
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # [batch_size, 1, hid_dim]

        # Вычисляем веса внимания
        attention = torch.bmm(v, energy).squeeze(1)  # [batch_size, src_len]

        # Применяем softmax для нормализации
        return torch.softmax(attention, dim=1)  # [batch_size, src_len]

# Декодер
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, attention):
        super(Decoder, self).__init__()
        self.output_dim = output_dim  # Размер словаря выходного языка
        self.attention = attention    # Механизм внимания
        self.embedding = nn.Embedding(output_dim, emb_dim)  # Слой эмбеддингов
        self.rnn = nn.GRU(hid_dim + emb_dim, hid_dim, batch_first=True)  # RNN (GRU)
        self.fc_out = nn.Linear(hid_dim * 2 + emb_dim, output_dim)  # Выходной полносвязный слой

    def forward(self, input, hidden, encoder_outputs):
        # input: [batch_size], hidden: [batch_size, hid_dim], encoder_outputs: [batch_size, src_len, hid_dim]

        # Добавляем измерение шага времени
        input = input.unsqueeze(1)  # [batch_size, 1]

        # Эмбеддинг текущего входного слова
        embedded = self.embedding(input)  # [batch_size, 1, emb_dim]

        # Вычисляем веса внимания
        a = self.attention(hidden, encoder_outputs)  # [batch_size, src_len]

        # Приводим форму для умножения
        a = a.unsqueeze(1)  # [batch_size, 1, src_len]

        # Вычисляем контекстный вектор как взвешенную сумму выходов кодера
        weighted = torch.bmm(a, encoder_outputs)  # [batch_size, 1, hid_dim]

        # Объединяем эмбеддинг и контекстный вектор
        rnn_input = torch.cat((embedded, weighted), dim=2)  # [batch_size, 1, emb_dim + hid_dim]

        # Пропускаем через RNN
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))  # output: [batch_size, 1, hid_dim], hidden: [1, batch_size, hid_dim]

        # Убираем измерение слоя
        hidden = hidden.squeeze(0)  # [batch_size, hid_dim]
        output = output.squeeze(1)  # [batch_size, hid_dim]
        embedded = embedded.squeeze(1)  # [batch_size, emb_dim]
        weighted = weighted.squeeze(1)  # [batch_size, hid_dim]

        # Предсказываем следующее слово
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))  # [batch_size, output_dim]

        return prediction, hidden, a.squeeze(1)  # Возвращаем предсказание, новое скрытое состояние и веса внимания

# Объединяем кодер и декодер в модель Seq2Seq
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder  # Кодер
        self.decoder = decoder  # Декодер

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: [batch_size, src_len], trg: [batch_size, trg_len]
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        # Тензор для хранения предсказаний
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(device)

        # Пропускаем входные данные через кодер
        encoder_outputs, hidden = self.encoder(src)  # encoder_outputs: [batch_size, src_len, hid_dim], hidden: [1, batch_size, hid_dim]
        hidden = hidden.squeeze(0)  # Приводим hidden к форме [batch_size, hid_dim]

        # Начинаем с токена <SOS>
        input = trg[:, 0]  # [batch_size]

        # Список для хранения весов внимания (необязательно)
        attentions = []

        # Проходим по каждому шагу времени в целевом предложении
        for t in range(1, trg_len):
            # Пропускаем через декодер
            output, hidden, attention = self.decoder(input, hidden, encoder_outputs)
            # Сохраняем предсказание
            outputs[:, t, :] = output
            # Решаем, использовать ли teacher forcing
            teacher_force = np.random.random() < teacher_forcing_ratio
            # Получаем слово с максимальной вероятностью
            top1 = output.argmax(1)
            # Определяем следующий вход для декодера
            input = trg[:, t] if teacher_force else top1
            # Сохраняем веса внимания (необязательно)
            attentions.append(attention.cpu().detach().numpy())

        return outputs, attentions  # Возвращаем все предсказания и веса внимания

# Создаём экземпляры кодера, декодера и модели Seq2Seq
attn = Attention(HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, attn)
model = Seq2Seq(enc, dec).to(device)

# === 4. Обучение модели ===

# Определяем функцию потерь и оптимизатор
criterion = nn.CrossEntropyLoss(ignore_index=english_vocab["<PAD>"])  # Игнорируем потери на позициях с <PAD>
optimizer = optim.Adam(model.parameters())

# Функция для обучения модели на одной эпохе
def train(model, dataloader, optimizer, criterion, clip):
    model.train()  # Устанавливаем модель в режим обучения
    epoch_loss = 0  # Инициализируем суммарную потерю

    for src, trg in dataloader:
        # Переносим данные на выбранное устройство
        src = src.to(device)
        trg = trg.to(device)

        optimizer.zero_grad()  # Обнуляем градиенты

        # Пропускаем данные через модель
        output, _ = model(src, trg)

        # Преобразуем выходы и целевые значения для функции потерь
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)  # Пропускаем первый токен <SOS>
        trg = trg[:, 1:].reshape(-1)

        # Вычисляем потерю
        loss = criterion(output, trg)

        # Обратное распространение ошибки
        loss.backward()

        # Ограничиваем градиенты
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Обновляем параметры
        optimizer.step()

        # Накопливаем потерю
        epoch_loss += loss.item()

    return epoch_loss / len(dataloader)  # Возвращаем среднюю потерю за эпоху

# Обучаем модель
N_EPOCHS = 1000  # Количество эпох
CLIP = 1         # Максимальная норма градиента

for epoch in range(N_EPOCHS):
    loss = train(model, dataloader, optimizer, criterion, CLIP)
    if (epoch + 1) % 100 == 0:
        print(f'Эпоха: {epoch + 1}, Потеря: {loss:.4f}')

# === 5. Тестирование модели ===

# Функция для перевода предложения с использованием обученной модели
def translate_sentence(model, sentence, english_vocab, french_vocab, max_len=10):
    model.eval()  # Устанавливаем модель в режим оценки

    # Преобразуем входное предложение в индексы
    tokens = sentence.split()
    indices = [english_vocab.get(token, english_vocab["<UNK>"]) for token in tokens]
    src_tensor = torch.LongTensor(indices).unsqueeze(0).to(device)  # [1, src_len]

    with torch.no_grad():
        encoder_outputs, hidden = model.encoder(src_tensor)
    hidden = hidden.squeeze(0)  # [1, hid_dim]

    # Начинаем с токена <SOS>
    trg_indices = [french_vocab["<SOS>"]]

    # Список для хранения весов внимания
    attentions = []

    for i in range(max_len):
        trg_tensor = torch.LongTensor([trg_indices[-1]]).to(device)  # [1]
        with torch.no_grad():
            output, hidden, attention = model.decoder(trg_tensor, hidden, encoder_outputs)
        pred_token = output.argmax(1).item()  # Получаем индекс с максимальной вероятностью
        trg_indices.append(pred_token)  # Добавляем предсказанный токен в результат
        attentions.append(attention.cpu().numpy())  # Сохраняем веса внимания

        if pred_token == french_vocab["<EOS>"]:
            break

    # Преобразуем индексы обратно в слова
    trg_tokens = [list(french_vocab.keys())[list(french_vocab.values()).index(idx)] for idx in trg_indices[1:]]

    return ' '.join(trg_tokens), attentions  # Возвращаем переведённое предложение и веса внимания

# Пример перевода
sentence = "hello how are you"
translation, attentions = translate_sentence(model, sentence, english_vocab, french_vocab)
print(f"Входное предложение: {sentence}")
print(f"Переведённое предложение: {translation}")

# === 6. Визуализация внимания (необязательно) ===

import matplotlib.pyplot as plt
import seaborn as sns

def display_attention(sentence, translation, attentions):
    # Преобразуем веса внимания в массив NumPy
    attention = np.array(attentions)
    attention = attention[:len(translation.split()), :len(sentence.split())]

    # Создаём тепловую карту
    plt.figure(figsize=(10, 8))
    sns.heatmap(attention, annot=True, cmap='viridis',
                xticklabels=sentence.split(),
                yticklabels=translation.split())
    plt.xlabel('Входное предложение')
    plt.ylabel('Переведённое предложение')
    plt.show()

# Вызов функции визуализации
display_attention(sentence, translation, attentions)


Эпоха: 100, Потеря: 0.0001


KeyboardInterrupt: 